In [1]:
#Generic
from collections import Counter
from collections import OrderedDict
import os
from os.path import join
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
import itertools
import datetime
from datetime import timedelta


#NLTK Stopword List
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('arabic')

#Gensim (LDA-Modelling)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaMulticore
from gensim.test.utils import datapath

#PDF-Reader
import pdfplumber

import pyLDAvis.gensim_models
import xml.etree.ElementTree as ETree

# plots
from bidi.algorithm import get_display
import arabic_reshaper
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
lda_model = LdaMulticore.load(r"C:\Users\kantg\Documents\Uni\nlp_arabic\saved_models\5_topics_pagewise_full")

with open('save_dict_pagewise_full.pkl', 'rb') as f:
    load_dat = pickle.load(f)
load_dat.keys()

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['data_df', 'pagewise_for_lda_chosen_tokens', 'id2word', 'corpus'])

In [3]:
df = load_dat['data_df']
df = df.sort_values(by='date')
df

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,page,file,date,cleaned_tokens,enemy_words_occurrences_counter,enemy_words_occurrences_sum
211,1 ـه1437/1/3 | لولأا ددعلا\nيملاعإ بتكم برقأ ع...,1,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[يثرب, محرم, برج, بتوفيق, الطوارئ, نوعية, معدا...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",2
213,3 ـه1437/1/3 | لولأا ددعلا\nّ\n)تيركت( لامشو )...,3,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[بيني, الكونغو, شمايل, موزمبيق, وأسفر, بوسط, ك...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
214,ـه1437/1/3 | لولأا ددعلا 4\nّ\n)ةروبصلا( ةيرق ...,4,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[لتدمريها, عزام, برئ, بثالثة, منفصلة, واستهدفو...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
215,5 ـه1437/1/3 | لولأا ددعلا\nّ\nّ\nيلشماقلا ةني...,5,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[محرم, بتوفيق, عيار, مسدس, بتوفيق, حوران, النو...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
216,ـه1437/1/3 | لولأا ددعلا 6\nّ\n)ةريفسلا(و )راج...,6,the-islamic-state-al-nabacc84_-newsletter-1.pdf,2015-10-16,"[وحربه, وأهله, دفعه, خان, طائفة, ردة, امتنعت, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
...,...,...,...,...,...,...,...
1782,7\nراــبخأ 315 ددعلا\nـه 1443 يناثلا عيبر 27 س...,7,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[وعاد, لتدمريها, بفرض, الكاملة, الكيلو, وجاء, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",4
1783,8\n315 ددعلا راــبخأ\nـه 1443 يناثلا عيبر 27 س...,8,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[سيطر, سابق, بندقية, اغتنمها, بيني, الكونغو, ب...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
1784,9\nتلااقم 315 ددعلا\nـه 1443 يناثلا عيبر 27 سي...,9,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[يرافقهم, تمشيط, وضابط, الحروب, مختار, محلية, ...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",0
1785,10\n315 ددعلا ديهش ةصق\nـه 1443 يناثلا عيبر 27...,10,The-Islamic-State-al-Nabā-Newsletter-315.pdf,2021-10-22,"[جمادى, بتوفيق, بادية, السلمية, إلعطابها, لتدم...","{'نصارى': 0, 'صليبيون': 0, 'صليبيين': 0, 'صليب...",5


In [4]:
# part df monthwise
entries_monthwise = []
for year in range(2015, 2022):
    for month in range(1,13):
        df_append = df[df['date'].apply(lambda x: x.year == year and x.month == month)]
        df_append.reset_index(drop=True, inplace=True)
        if not df_append.empty: 
            entries_monthwise.append(df_append)

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# new dfs with data by month and issue 
df_by_issue = pd.DataFrame(columns=['file', 'cleaned_tokens','date'])
df_by_month = pd.DataFrame(columns=['cleaned_tokens','date'])
for df in entries_monthwise:
    for name, date in zip(list(OrderedDict.fromkeys(df['file'])), list(OrderedDict.fromkeys(df['date']))): # ordered dict instead of set to keep order
        row = {'file': name , 'cleaned_tokens': list(itertools.chain(*df[df['date'] == date]['cleaned_tokens'].tolist())), 'date': date}
        df_by_issue = df_by_issue.append(row, ignore_index=True)
    row = {'cleaned_tokens': list(itertools.chain(*df['cleaned_tokens'].tolist())), 'date': datetime.datetime.strptime(df['date'][0].strftime('%Y-%m'), '%Y-%m')}
    df_by_month = df_by_month.append(row, ignore_index=True)
        

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df_by_issue[:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,cleaned_tokens,date
0,the-islamic-state-al-nabacc84_-newsletter-1.pdf,"[يثرب, محرم, برج, بتوفيق, الطوارئ, نوعية, معدا...",2015-10-16
1,the-islamic-state-al-nabacc84_-newsletter-2.pdf,"[الزهور, قذيفة, الحجر, وأشارت, الواردة, حيي, و...",2015-10-23
2,the-islamic-state-al-nabacc84_-newsletter-3.pdf,"[المناطق, المحيطة, وتكريت, شرعي, فأعلنوا, اإلس...",2015-10-30
3,the-islamic-state-al-nabacc84_-newsletter-4.pdf,"[احصل, المواد, أقرب, ضعفا, أضعاف, وقطر, اإلسال...",2015-11-06
4,the-islamic-state-al-nabacc84_-newsletter-5.pdf,"[البدري, إماما, بكونه, محققا, الفتن, اليقني, ص...",2015-11-13
5,the-islamic-state-al-nabacc84_-newsletter-6.pdf,"[شبل, أشبال, بالشكر, حقا, نرص, وسالم, ختم, أبي...",2015-11-20
6,the-islamic-state-al-nabacc84_-newsletter-7.pdf,"[لهالك, للرجل, الشافعي, ربيع, الفرج, الكرب, ال...",2015-11-27
7,the-islamic-state-al-nabacc84_-newsletter-8.pdf,"[فانظر, فإنها, تخلو, ماجه, للزوجة, بمجرد, كراه...",2015-12-04
8,the-islamic-state-al-nabacc84_-newsletter-9.pdf,"[مخاطر, فشل, الحملة, الفلوجة, بقيادة, حيدر, ال...",2015-12-11
9,the-islamic-state-22al-nibacc84-newsletter-10.pdf,"[حتفه, آخران, شوال, لعدد, دراجات, السعدية, بلد...",2015-12-18


In [7]:
df_by_issue['date'][:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     2015-10-16
1     2015-10-23
2     2015-10-30
3     2015-11-06
4     2015-11-13
5     2015-11-20
6     2015-11-27
7     2015-12-04
8     2015-12-11
9     2015-12-18
10    2015-12-25
11    2016-01-01
12    2016-01-08
13    2016-01-15
14    2016-01-22
15    2016-01-29
16    2016-02-05
17    2016-02-12
18    2016-02-19
19    2016-02-26
Name: date, dtype: object

In [8]:
df_by_month['date'][:20]

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    2015-10-01
1    2015-11-01
2    2015-12-01
3    2016-01-01
4    2016-02-01
5    2016-03-01
6    2016-04-01
7    2016-05-01
8    2016-06-01
9    2016-07-01
10   2016-08-01
11   2016-09-01
12   2016-10-01
13   2016-11-01
14   2016-12-01
15   2017-01-01
16   2017-02-01
17   2017-03-01
18   2017-04-01
19   2017-05-01
Name: date, dtype: datetime64[ns]

In [9]:
df_by_month

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cleaned_tokens,date
0,"[يثرب, محرم, برج, بتوفيق, الطوارئ, نوعية, معدا...",2015-10-01
1,"[احصل, المواد, أقرب, ضعفا, أضعاف, وقطر, اإلسال...",2015-11-01
2,"[فانظر, فإنها, تخلو, ماجه, للزوجة, بمجرد, كراه...",2015-12-01
3,"[عقيد, محل, صفر, وبتوفيق, عناصره, أرسوا, بادية...",2016-01-01
4,"[الدقيق, حبة, قصرية, الشعري, حبة, طويلة, الحنط...",2016-02-01
...,...,...
67,"[حرارية, استهدافه, الحنطة, شمايل, علياوة, بعمل...",2021-05-01
68,"[مرافقيه, مام, تعرضت, للعديد, تسببت, وتحديدا, ...",2021-07-01
69,"[حاولت, دخول, استمرت, لدورية, الفرار, استهدفوه...",2021-08-01
70,"[شعبان, مدار, وأعطب, نفذه, جرس, الوادي, وعدد, ...",2021-09-01


In [10]:
# estimate topic distribution for the issues / monthwise

def estimate_topic_distr(series, lda_model):
    distr_df = pd.DataFrame(columns=['1', '2', '3', '4', '5'])
    corpus = [load_dat['id2word'].doc2bow(text) for text in series]
    for doc in corpus:
        topic_distr_vec = lda_model[doc][0]
        distr_df = distr_df.append({'1': topic_distr_vec[0][1], '2': topic_distr_vec[1][1], '3': topic_distr_vec[2][1], '4': topic_distr_vec[3][1], '5': topic_distr_vec[4][1]}, ignore_index=True)
    return distr_df

# concat the topic distributions:
df_by_issue = pd.concat([df_by_issue, estimate_topic_distr(df_by_issue['cleaned_tokens'], lda_model)], axis=1)
df_by_month = pd.concat([df_by_month, estimate_topic_distr(df_by_month['cleaned_tokens'], lda_model)], axis=1)

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df_by_month

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cleaned_tokens,date,1,2,3,4,5
0,"[يثرب, محرم, برج, بتوفيق, الطوارئ, نوعية, معدا...",2015-10-01,0.086028,0.480679,0.235408,0.091138,0.106748
1,"[احصل, المواد, أقرب, ضعفا, أضعاف, وقطر, اإلسال...",2015-11-01,0.182979,0.169221,0.255383,0.199765,0.192652
2,"[فانظر, فإنها, تخلو, ماجه, للزوجة, بمجرد, كراه...",2015-12-01,0.232038,0.157030,0.264226,0.202921,0.143786
3,"[عقيد, محل, صفر, وبتوفيق, عناصره, أرسوا, بادية...",2016-01-01,0.210812,0.159622,0.221808,0.133682,0.274076
4,"[الدقيق, حبة, قصرية, الشعري, حبة, طويلة, الحنط...",2016-02-01,0.239131,0.160682,0.261566,0.168551,0.170069
...,...,...,...,...,...,...,...
67,"[حرارية, استهدافه, الحنطة, شمايل, علياوة, بعمل...",2021-05-01,0.248584,0.105499,0.196959,0.147958,0.301000
68,"[مرافقيه, مام, تعرضت, للعديد, تسببت, وتحديدا, ...",2021-07-01,0.191843,0.171376,0.345625,0.185046,0.106109
69,"[حاولت, دخول, استمرت, لدورية, الفرار, استهدفوه...",2021-08-01,0.159416,0.149194,0.284361,0.080601,0.326428
70,"[شعبان, مدار, وأعطب, نفذه, جرس, الوادي, وعدد, ...",2021-09-01,0.122648,0.139502,0.347320,0.159527,0.231004


In [14]:
fig = go.Figure()
for i in range(1,6):
    fig.add_trace(go.Scatter(x=df_by_month['date'], y=df_by_month[str(i)],
                             mode='lines',
                             name='Topic ' + str(i)))
fig.update_layout(title='Topic Distribution over time for Tokens aggregated by month',
                  xaxis = dict(title='Time',
                            tick0 = df_by_month['date'].iloc[0],
                            tickvals = df_by_month['date'],
                            #ticktext = df_by_month['date']
                            ),
                  yaxis_title='Topic Proportion')
fig.write_html("plotly_topic_distr_monthwise.html")
fig.show()

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [13]:
fig = go.Figure()
for i in range(1,6):
    fig.add_trace(go.Scatter(x=df_by_issue['date'], y=df_by_issue[str(i)],
                             mode='lines',
                             name='Topic ' + str(i)))
fig.update_layout(title='Topic Distribution over time for Tokens aggregated by month',
                  xaxis = dict(title='Time',
                            tick0 = df_by_month['date'].iloc[0],
                            tickvals = df_by_month['date'],
                            dtick = 1
                            #ticktext = df_by_month['date']
                            ),
                  yaxis_title='Topic Proportion')
fig.write_html("plotly_topic_distr_issuewise.html")
fig.show()

C:\Users\kantg\.conda\envs\arabic-nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

